In [ ]:
!pip3 list | grep -i tensorrt

In [2]:
import torch

OSError: libcurand.so.10: cannot open shared object file: No such file or directory

In [1]:
import torch

from torchvision import models, transforms

OSError: libcurand.so.10: cannot open shared object file: No such file or directory

In [ ]:
# resnet50 with 1000 classification classes 
model = models.resnet50(pretrained=True).to("cuda")


In [ ]:
from PIL import Image 
img = Image.open("./IMG_3522.jpeg")
img

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
img = transform(img)
img.shape

In [ ]:
img_batch = torch.unsqueeze(img, 0).to("cuda")
img_batch.shape

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(img_batch)
probs = torch.nn.functional.softmax(outputs[0], dim=0)
probs

In [ ]:
import pandas as pd 

categories = pd.read_csv("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt" , header=None)
categories[0][1]

In [ ]:
topk = 5
prob, class_number = torch.topk(probs, topk)
for i in range(topk):
    probability = prob[i].item()
    class_name = categories[0][int(class_number[i])]
    print("{}% {}".format(int(probability*100), class_name))
    

In [ ]:
#copy from TRT
import time
import numpy as np

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

def benchmark(model, device="cuda", input_shape=(32, 3, 224, 224), dtype='fp32', nwarmup=10, nruns=30):
    input_data = torch.randn(input_shape)
    input_data = input_data.to(device)
        
    print("Warm up ...")
    with torch.no_grad():
        for _ in range(nwarmup):
            print('processing')
            features = model(input_data)
    torch.cuda.synchronize()
    print("Start timing ...")
    timings = []
    with torch.no_grad():
        for i in range(1, nruns+1):
            start_time = time.time()
            features = model(input_data)
            torch.cuda.synchronize()
            end_time = time.time()
            timings.append(end_time - start_time)
            if i%10==0:
                print('Iteration %d/%d, ave batch time %.2f ms'%(i, nruns, np.mean(timings)*1000))

    print("Input shape:", input_data.size())
    print("Output features size:", features.size())
    print('Average batch time: %.2f ms'%(np.mean(timings)*1000))

In [ ]:
#CPU benchmarks
# model = model.to("cpu")
# benchmark(model)
benchmark(model, device="cpu")

In [ ]:
#CUDA benchmarks
model = model.to("cuda")
benchmark(model)

In [ ]:
traced_model = torch.jit.trace(model, [torch.randn((8, 3, 224, 224)).to("cuda")])

In [ ]:
!pip3 install tensorrt-5.0.2.6-py2.py3-none-any.whl  
!pip3 install pycuda

In [ ]:
import torch_tensorrt

trt_model = torch_tensorrt.compile(
    traced_model,
    inputs = [torch_tensorrt.Input((32, 3, 224, 224), dtype=torch.float32)],
    enabled_precisions = {torch.float32}
)

In [ ]:
benchmark(trt_model)

In [ ]:
trt_model.eval()
with torch.no_grad():
    outputs = trt_model(img_batch)
prob = torch.nn.functional.softmax(outputs[0], dim=0)

probs, classes = torch.topk(prob, topk)
for i in range(topk):
    probability = probs[i].item()
    class_label = categories[0][int(classes[i])]
    print("%{} {}".format(int(probability*100), class_label))